In [ ]:
!pip install torch torchvision matplotlib pillow

In [2]:
import torch
import torchvision
import torch.nn as nn
import itertools
from torch.optim import Adam
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import os
from torch.utils.data import DataLoader, TensorDataset, Dataset
from tqdm import tqdm
from MyTensorDataset import MyTensorDataset


## Hyperparameters

In [4]:
num_epochs = 1000
batch_size = 10 #64

In [5]:
# Define path to dataset
data_path = r'.\0_layers'

In [6]:
# Create the dataset
dataset = MyTensorDataset(data_path)
classes = ('black', 'cat')

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers = 4, persistent_workers=True)

# Example: iterating through the DataLoader
#for batch_tensors, batch_labels in dataloader:
#    print(batch_tensors)  # The batch tensors
#    print(batch_labels)   # The corresponding labels for each tensor
asd = []
for t,l in dataset:
    if l == 0:
        asd.append(l)
print(len(asd))

#a= []
#b=[]
#for batch_index, (real_A, real_B) in enumerate(dataloader):
#    a.append(real_A)
#    b.append(real_B)
#print(len(a))
#print(len(b))
#print(a[0].shape)
#print(b[0])

2749


## Setup device

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## Define Model

In [8]:
# Generator (U-Net inspired)
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.main(x)

# Discriminator (PatchGAN)
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv3d(64, 1, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.main(x)


## Loss Function

In [9]:
#Adversarial Loss: Encourages generators to produce realistic images.
#Cycle-Consistency Loss: Ensures the transformation is reversible.
#Identity Loss (Optional): Preserves key features of the input.
adversarial_loss = nn.MSELoss()
cycle_loss = nn.L1Loss()
identity_loss = nn.L1Loss()

## Training Loop

In [ ]:
# Initialize models
G_A2B = Generator().to(device)
G_B2A = Generator().to(device)
D_A = Discriminator().to(device)
D_B = Discriminator().to(device)

# Optimizers
optimizer_G = Adam(itertools.chain(G_A2B.parameters(), G_B2A.parameters()), lr=0.00002, betas=(0.5, 0.999))
optimizer_D_A = Adam(D_A.parameters(), lr=0.00002, betas=(0.5, 0.999))
optimizer_D_B = Adam(D_B.parameters(), lr=0.00002, betas=(0.5, 0.999))

# Training loop
for epoch in range(num_epochs):
    print(f"Epoch [{epoch +1 }/{num_epochs}]")
    real_A = []
    real_B = []
    for batch_index, (data,labels) in enumerate(tqdm(dataloader)):
        #print(len(batch_data))
        #print(len(batch_data[0]),batch_data[1])
        real_A = data[labels == 0]  # Data for domain A
        real_B = data[labels == 1]
        
        #print(len(real_A),len(real_B))
        real_A, real_B = real_A.to(device), real_B.to(device)
        #print(real_A.shape)
        # Train Generators
        optimizer_G.zero_grad()

        # GAN loss
        fake_B = G_A2B(real_A)
        fake_A = G_B2A(real_B)
        loss_G_A2B = adversarial_loss(D_B(fake_B), torch.ones_like(D_B(fake_B)))
        loss_G_B2A = adversarial_loss(D_A(fake_A), torch.ones_like(D_A(fake_A)))

        # Cycle-consistency loss
        reconstructed_A = G_B2A(fake_B)
        reconstructed_B = G_A2B(fake_A)
        loss_cycle_A = cycle_loss(reconstructed_A, real_A)
        loss_cycle_B = cycle_loss(reconstructed_B, real_B)

        # Total generator loss
        loss_G = loss_G_A2B + loss_G_B2A + 10 * (loss_cycle_A + loss_cycle_B)
        loss_G.backward()
        optimizer_G.step()

        # Train Discriminators
        optimizer_D_A.zero_grad()
        loss_D_A = (adversarial_loss(D_A(real_A), torch.ones_like(D_A(real_A))) +
                    adversarial_loss(D_A(fake_A.detach()), torch.zeros_like(D_A(fake_A.detach())))) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        optimizer_D_B.zero_grad()
        loss_D_B = (adversarial_loss(D_B(real_B), torch.ones_like(D_B(real_B))) +
                    adversarial_loss(D_B(fake_B.detach()), torch.zeros_like(D_B(fake_B.detach())))) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()

    print(f"Epoch {epoch}/{num_epochs}, Loss G: {loss_G.item()}, Loss D_A: {loss_D_A.item()}, Loss D_B: {loss_D_B.item()}")


Epoch [1/2000]


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:10<00:00, 42.12it/s]


Epoch 0/2000, Loss G: 1.7471169233322144, Loss D_A: 0.2464035302400589, Loss D_B: 0.24848325550556183
Epoch [2/2000]


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:10<00:00, 43.63it/s]


Epoch 1/2000, Loss G: 1.5622472763061523, Loss D_A: 0.2455386221408844, Loss D_B: 0.2448953092098236
Epoch [3/2000]


...

100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:10<00:00, 43.27it/s]


Epoch 46/2000, Loss G: 0.6251744627952576, Loss D_A: 0.23063822090625763, Loss D_B: 0.23942820727825165
Epoch [48/2000]


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:10<00:00, 42.26it/s]


Epoch 47/2000, Loss G: 0.618886411190033, Loss D_A: 0.23581302165985107, Loss D_B: 0.233137309551239
Epoch [49/2000]


 54%|███████████████████████████████████████████▏                                    | 243/450 [00:06<00:05, 36.03it/s]

## Save Models and Visualise results

In [181]:
torch.save(G_A2B.state_dict(), 'generator_A2B.pth')
torch.save(G_B2A.state_dict(), 'generator_B2A.pth')


In [186]:
layer = 0
model = 0
PATH = f'./black_layers_permuted/0/0_0000.pth'
real_A = torch.load(f'./black_layers_permuted/{layer}/{layer}_{model:04}.pth', weights_only=True).to(device)
real_B = torch.load(f'./cat_layers_permuted/{layer}/{layer}_{model:04}.pth', weights_only=True).to(device)


# Assuming G_A and G_B are your trained generators
G_A2B.to(device).eval()
G_B2A.to(device).eval()

# Generate fake images (domain A -> B and B -> A)
fake_B = G_A2B(real_A).permute(1, 0, 2, 3)
fake_A = G_B2A(real_B).permute(1, 0, 2, 3)


In [1]:
def visTensor(tensor, ch=0, allkernels=False, nrow=8, padding=1): 
        n,c,w,h = tensor.shape

        if allkernels: tensor = tensor.view(n*c, -1, w, h)
        elif c != 3: tensor = tensor[:,ch,:,:].unsqueeze(dim=1)

        rows = np.min((tensor.shape[0] // nrow + 1, 64))    
        grid = torchvision.utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
        plt.figure( figsize=(nrow,rows) )
        plt.imshow(grid.numpy().transpose((1, 2, 0)))

print("A")
visTensor(real_A.cpu().permute(1, 0, 2, 3), ch=0, allkernels=False)
visTensor(fake_A.cpu(), ch=0, allkernels=False)

print("B")
visTensor(real_B.cpu().permute(1, 0, 2, 3), ch=0, allkernels=False)
visTensor(fake_B.cpu(), ch=0, allkernels=False)

A


NameError: name 'real_A' is not defined